Test mpremote (command) API with some basic commands

In [1]:
import argparse  # needed as some of the commands require arguments to be passed

import mpremote
import mpremote.commands as mprc  # shortcut to the commands module
from mpremote.main import State  # state is needed to handle the mpremote state of a device

In [2]:
# State hanldes the state of the device - currently mostly serial
mcu_1_state = State()
mpc.do_resume(mcu_1_state)  # no soft reste at start of session

devices = mpc.do_connect(mcu_1_state, argparse.Namespace(device=["list"]))


# the output of this is a list of devices that are connected
# BUG: the outut is printed to stdout instead of being returned

# for an API a function that returns a list of devices would be useful
# mpremote.api.get_devices()
# something like the below ( )

# output = mpc.get_device_list()
# print("Connected devices:")
# for device in output:
#     print(device)

COM12 None 0000:0000 Microsoft None
COM13 None 0000:0000 Microsoft None
COM24 206437A1304E f055:9800 Microsoft None
COM31 18:8B:0E:03:1B:54 303a:1001 Microsoft None


In [3]:
# connecting to the first device in the list is a bit clumsy as well as it requires the device name 'auto' to be passed as a string
#
args = argparse.Namespace(device=["auto"])

try:
    mpc.do_connect(mcu_1_state, args)
    # nothing is returned, so unclear if / how it can be determined that a connection attemp succeeded
    if mcu_1_state.transport:
        print(f"{mcu_1_state.transport.device_name=}")
except Exception as e:
    print(e)

# Also unclear how to determine if a connection attempt failed

mcu_1_state.transport.device_name='COM24'


In [4]:
print(mcu_1_state)
# the state itself does not provide any information about the connection
# in order to get information about the connection, the state must be accessed via the transport attribute
# however, there it is not clear what attributes can be accessed reliably across all connection types

if mcu_1_state.transport:
    print(f"{mcu_1_state.transport.device_name=}")
    print(f"{mcu_1_state.transport.mounted=}")
    print(f"{mcu_1_state.transport.in_raw_repl=}")
    print(f"{mcu_1_state.transport.serial=}")
else:
    print("No connection")

# it would make sense to reflect some of the transport's attributes in the state itself
# e.g.
# mpremote.state.connected
# mpremote.state.device_name


#     @property
#     def device_name(self) -> str:
#         return "" if not self.transport else self.transport.device_name

mcu_1_state.transport.device_name='COM24'
mcu_1_state.transport.mounted=False
mcu_1_state.transport.in_raw_repl=False
mcu_1_state.transport.serial=Serial<id=0x21e90d3dde0, open=True>(port='COM24', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)


In [5]:
# mcu_1_state.device_name

In [6]:
# try to create a 2nd connection to the same device - tis should raise an error on Windows (but not on Linux)

mcu_2_state = mpremote.main.State()
mpc.do_resume(mcu_2_state)  # no soft reste at start of session

# args = argparse.Namespace(device=["port:COM6"])
args = argparse.Namespace(device=[mcu_1_state.transport.device_name])

try:
    mpc.do_connect(mcu_2_state, args)
# except (mpc.TransportError, mpc.CommandError) as e:
except mpc.CommandError as e:
    print(type(e), e)

# I would expect a transport error, but instead a command error is raised

<class 'mpremote.commands.CommandError'> failed to access COM24 (it may be in use by another program)


In [7]:
# discnnect MCU1
mpc.do_disconnect(mcu_1_state)

In [10]:
try:
    mpc.do_connect(mcu_2_state, args)
    print("Connection succeeded")
    if mcu_2_state.transport:
        print(f"{mcu_2_state.transport.device_name=}")
# except (mpc.TransportError, mpc.CommandError) as e:
except mpc.CommandError as e:
    print(type(e), e)

Connection succeeded
mcu_2_state.transport.device_name='COM24'


In [11]:
mpc.do_resume(mcu_2_state)

In [12]:
mpc.do_soft_reset(mcu_2_state)
# reset runs quite a bit faster than via the cmd line
# 22.6 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

In [13]:
# Set and get time works as expected
# using the argparse.Namespace(set=True) is a bit verbose to pass just a boolean
mpc.do_rtc(mcu_2_state, argparse.Namespace(set=True))
mpc.do_rtc(mcu_2_state, argparse.Namespace(set=False))

(2025, 3, 16, 6, 23, 56, 25, 245)


In [14]:
comport = mcu_2_state.transport.device_name
# mpc.do_disconnect(mcu_2_state)

# expilitly disconnect from COM17
mpc.do_disconnect(mcu_2_state, argparse.Namespace(device=[f"port:{comport}"]))

In [32]:
mpc.do_mount(mcu_1_state, argparse.Namespace(path=["."], unsafe_links=False))

# Mount work as expected,
# BUG: but the output is printed to stdout instead of being returned

Local directory . is mounted at /remote


In [16]:
for verb in [True, False]:
    mpc.do_filesystem(mcu_1_state, argparse.Namespace(command=["ls"], path=["/remote"], verbose=verb))
    print("-" * 10)

mpc.do_filesystem(mcu_1_state, argparse.Namespace(command=["ls"], path=["/"], verbose=True))

# BUG: the output is printed to stdout instead of being returned

ls :/remote
           0 board_id.ipynb
           0 libusb_flash.ipynb
           0 mpr_command.ipynb
----------
           0 board_id.ipynb
           0 libusb_flash.ipynb
           0 mpr_command.ipynb
----------
ls :/
           0 flash/
           0 sd/
           0 remote/


In [22]:
from pathlib import Path

mpc.do_run(mcu_1_state, argparse.Namespace(path=["D:\\mypython\\mpflash\\mpflash\\mpremoteboard\\mpy_fw_info.py"], follow=True))

# mpc.do_run(
#     mcu_1_state, argparse.Namespace(path=[Path("mpflash/mpremoteboard/mpy_fw_info.py").absolute()], follow=True)
# )


# File C:\develop\MyPython\micropython\tools\mpremote\mpremote\commands.py:397, in _do_execbuffer..stdout_write_bytes(b)
#     395 def stdout_write_bytes(b):
#     396     b = b.replace(b"\x04", b"")
# --> 397     sys.stdout.buffer.write(b)
#     398     sys.stdout.buffer.flush()

# AttributeError: 'OutStream' object has no attribute 'buffer'

# BUG: output is printed to stdout instead of being returned

{'cpu': 'STM32F405RG', 'build': '389', 'board': 'PYBv1.1 with STM32F405RG', 'family': 'micropython', '_build': 'PYBV11-DP_THREAD', 'ver': 'v1.25.0-preview-389', 'port': 'stm32', 'arch': 'armv7emsp', 'mpy': 'v6.3', 'version': '1.25.0-preview'}


In [24]:
x = mpc.do_eval(mcu_1_state, argparse.Namespace(expr=["1+1"]))
print(x)

# bug in do_eval: the output is printed to stdout instead of being returned
# File C:\develop\MyPython\micropython\tools\mpremote\mpremote\commands.py:397, in _do_execbuffer..stdout_write_bytes(b)
#     395 def stdout_write_bytes(b):
#     396     b = b.replace(b"\x04", b"")
# --> 397     sys.stdout.buffer.write(b)
#     398     sys.stdout.buffer.flush()

# AttributeError: 'OutStream' object has no attribute 'buffer'

2
None


In [43]:
# bit of a different API - this is simpler to use
mpc.do_umount(mcu_1_state, path="/remote")

In [ ]:
# mpc.do_edit(mcu_1_state, argparse.Namespace(path=["/remote/main.py"]))

In [26]:
# disconnect and reconnect
mpc.do_disconnect(mcu_1_state)

In [27]:
# %%timeit
# 819 ms ± 69.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
! mpremote ls

ls :
          14 foo.txt
           0 folder_1/
           0 p1_meter/
           0 lib/
         628 test.py
         104 board_info.toml
           0 System Volume Information/
      455656 MTCPBWY-v1.gcode


In [ ]:
# %%timeit
# 88.3 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
mpc.do_connect(mcu_1_state, argparse.Namespace(device=["port:COM6"]))
mpc.do_filesystem(mcu_1_state, argparse.Namespace(command=["ls"], path=["/tests"], verbose=True))

In [ ]:
mpc.do_filesystem_cp(
    mcu_1_state,
    src="C:\\develop\\MyPython\\micropython\\tests\\basics\\0prelim.py",
    dest=":0prelim.py",
    multiple=False,
    check_hash=False,
)
# works - but no return value - should return 0 or error code

mpc.do_filesystem_cp(
    mcu_1_state,
    src="C:\\develop\\MyPython\\micropython\\tests\\basics\\0prelim.py",
    dest=":0prelim.py",
    multiple=False,
    check_hash=True,
)
# works - but no return value - should return 0 or error code
# prints to tdout instead of returning

In [42]:
mpc.do_filesystem_recursive_cp(
    mcu_1_state,
    src="D:\\mypython\\mpflash\\scripts",
    dest=":\\sd",
    multiple=True,
    check_hash=True,
)

CommandError: cp: destination does not exist